# WEB Page Summarizer

In [ ]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2:1b"

In [ ]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [ ]:
import requests
from bs4 import BeautifulSoup
from bs4 import Comment
import time
from urllib.parse import urlparse
from requests.exceptions import RequestException

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
}

class Website:
    def __init__(self, url, timeout=10, delay=1):
        """
        Create this Website object from the given url using the BeautifulSoup library
        
        Args:
            url (str): The website URL to scrape
            timeout (int): Request timeout in seconds
            delay (int): Delay between requests to be polite
        """
        self.url = url
        self.timeout = timeout
        
        # Initialize all attributes with default values
        self.title = "No title found"
        self.text = "No content available"
        self.domain = None
        self.soup = None
        self.response = None
        self.metadata = {}
        
        # Be polite - add delay between requests
        time.sleep(delay)
        
        self._fetch_url()
        if self.response and self.response.status_code == 200:
            self._parse_content()
    
    def _fetch_url(self):
        """Fetch the URL with error handling"""
        try:
            self.response = requests.get(
                self.url, 
                headers=headers, 
                timeout=self.timeout,
                allow_redirects=True
            )
            self.response.raise_for_status()
            
            # Extract domain for reference
            parsed_url = urlparse(self.url)
            self.domain = parsed_url.netloc
            
        except RequestException as e:
            print(f"Error fetching {self.url}: {e}")
            self.response = None
    
    def _parse_content(self):
        """Parse the HTML content"""
        try:
            self.soup = BeautifulSoup(self.response.content, 'html.parser')
            self._extract_title()
            self._clean_content()
            self._extract_text()
            self._extract_metadata()
        except Exception as e:
            print(f"Error parsing content from {self.url}: {e}")
    
    def _extract_title(self):
        """Extract and clean the page title"""
        try:
            if self.soup.title and self.soup.title.string:
                self.title = self.soup.title.string.strip()
            else:
                # Try to find title in h1 if no title tag
                h1 = self.soup.find('h1')
                self.title = h1.get_text().strip() if h1 else "No title found"
        except Exception as e:
            print(f"Error extracting title: {e}")
            self.title = "Error extracting title"
    
    def _clean_content(self):
        """Remove irrelevant elements from the page"""
        if not self.soup or not self.soup.body:
            return
            
        try:
            # Elements to remove
            elements_to_remove = [
                "script", "style", "img", "input", "button", 
                "nav", "footer", "header", "aside", "form"
            ]
            
            for element in elements_to_remove:
                for tag in self.soup.body.find_all(element):
                    tag.decompose()
            
            # Remove comments
            for comment in self.soup.body.find_all(string=lambda text: isinstance(text, Comment)):
                comment.extract()
        except Exception as e:
            print(f"Error cleaning content: {e}")
    
    def _extract_text(self):
        """Extract and clean text content"""
        if not self.soup or not self.soup.body:
            self.text = "No content found"
            return
            
        try:
            # Get text with better formatting
            text = self.soup.body.get_text(separator="\n", strip=True)
            
            # Clean up excessive whitespace
            lines = [line.strip() for line in text.splitlines() if line.strip()]
            self.text = '\n'.join(lines) if lines else "No text content found"
        except Exception as e:
            print(f"Error extracting text: {e}")
            self.text = "Error extracting text content"
    
    def _extract_metadata(self):
        """Extract additional metadata"""
        self.metadata = {}
        
        try:
            # Meta description
            meta_desc = self.soup.find('meta', attrs={'name': 'description'})
            self.metadata['description'] = meta_desc.get('content') if meta_desc else None
            
            # Keywords
            meta_keywords = self.soup.find('meta', attrs={'name': 'keywords'})
            self.metadata['keywords'] = meta_keywords.get('content') if meta_keywords else None
            
            # Language
            html_tag = self.soup.find('html')
            self.metadata['language'] = html_tag.get('lang') if html_tag else None
        except Exception as e:
            print(f"Error extracting metadata: {e}")
    
    def get_links(self):
        """Extract all links from the page"""
        if not self.soup:
            return []
        
        links = []
        try:
            for link in self.soup.find_all('a', href=True):
                links.append({
                    'text': link.get_text(strip=True),
                    'url': link['href'],
                    'title': link.get('title', '')
                })
        except Exception as e:
            print(f"Error extracting links: {e}")
        
        return links
    
    def get_status_info(self):
        """Get HTTP status information"""
        if not self.response:
            return {"error": "No response available"}
        
        return {
            "status_code": self.response.status_code,
            "content_type": self.response.headers.get('content-type', ''),
            "content_length": len(self.response.content),
            "encoding": self.response.encoding
        }
    
    def is_successful(self):
        """Check if the website was successfully fetched and parsed"""
        return self.response is not None and self.response.status_code == 200
    
    def __str__(self):
        """String representation of the Website object"""
        return f"Website(url='{self.url}', title='{self.title}')"
    
    def __repr__(self):
        return self.__str__()

# Usage example
# if __name__ == "__main__":
#     # Example usage
#     try:
#         site = Website("https://example.com")
        
#         print(f"Title: {site.title}")
#         print(f"Domain: {site.domain}")
#         print(f"Status: {site.get_status_info()}")
#         print(f"Text length: {len(site.text)} characters")
#         print(f"Successful: {site.is_successful()}")
        
#         # Get first few links
#         links = site.get_links()[:5]
#         print("Sample links:")
#         for link in links:
#             print(f"  - {link['text']}: {link['url']}")
            
#     except Exception as e:
#         print(f"Failed to process website: {e}")



In [ ]:
# Usage example
if __name__ == "__main__":
    # Example usage
    try:
        site = Website("https://www.geeksforgeeks.org/blogs/interview-preparation/")
        
        print(f"Title: {site.title}")
        print(f"Domain: {site.domain}")
        print(f"Status: {site.get_status_info()}")
        print(f"Text length: {len(site.text)} characters")
        
        # Get first few links
        links = site.get_links()[:5]
        print("Sample links:")
        for link in links:
            print(f"  - {link['text']}: {link['url']}")
            
    except Exception as e:
        print(f"Failed to process website: {e}")

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    user_prompt += website.domain
    return user_prompt

In [ ]:
print(user_prompt_for(site))

In [ ]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
# Try this out, and then try for a few more websites

messages_for(site)

In [ ]:
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [ ]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://www.geeksforgeeks.org/blogs/interview-preparation/")